In [2]:
pip install datasets transformers

In [3]:
pip install accelerate -U

In [4]:
!pip install huggingface_hub

from huggingface_hub import notebook_login

notebook_login()

In [5]:
from datasets import load_dataset, Dataset
from transformers import Wav2Vec2ForSequenceClassification, Wav2Vec2Config, Wav2Vec2FeatureExtractor, Trainer, TrainingArguments
from torch.utils.data import DataLoader
from tqdm import tqdm
import pandas as pd
import numpy as np

In [6]:
dataset = load_dataset("mozilla-foundation/common_voice_13_0", "en", split="train", streaming=True, trust_remote_code=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [7]:
train_inputs = []
sampling_rates = []
num_load = 10000
for i, data in enumerate(dataset):
  if i < num_load:
    train_inputs.append(data['audio']['array'])
    sampling_rates.append(data['audio']['sampling_rate'])
  else:
    break

Reading metadata...: 1013968it [00:15, 66535.69it/s]


In [9]:
feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained("facebook/wav2vec2-base", sampling_rate=sampling_rates[0])

In [ ]:
inputs = feature_extractor(train_inputs, padding=True, sampling_rate=sampling_rates[0])

In [ ]:
train_data = {
    'input_values': inputs.input_values,
    'labels': np.random.randint(low=0, high=3, size=num_load)
}

In [ ]:
loaded_data = Dataset.from_dict(train_data)

In [ ]:
batch_size = 64
lr = 1e-4
num_epochs = 1

train_args = TrainingArguments(
    output_dir="./output",
    overwrite_output_dir=True,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    learning_rate=lr,
    num_train_epochs=num_epochs,
    evaluation_strategy="epoch",
    save_strategy="no"
    )

In [ ]:
config = Wav2Vec2Config(
)

In [ ]:
model = Wav2Vec2ForSequenceClassification.from_pretrained("facebook/wav2vec2-base", num_labels=3)

pretrainer = Trainer(
    model,
    args=train_args,
    train_dataset=loaded_data,
    eval_dataset=loaded_data
    )

In [ ]:
pretrainer.train()

Epoch,Training Loss,Validation Loss
1,No log,1.078046


TrainOutput(global_step=4, training_loss=1.1158828735351562, metrics={'train_runtime': 38.0626, 'train_samples_per_second': 0.841, 'train_steps_per_second': 0.105, 'total_flos': 7561577360535552.0, 'train_loss': 1.1158828735351562, 'epoch': 1.0})

In [ ]:
# df = pd.DataFrame(dataviewer)
# df.to_csv('/content/drive/MyDrive/244 Final Project/samples.csv')